In [0]:
%python
datalake_key = dbutils.secrets.get(scope="license_key", key="datalake-key")

spark.conf.set("fs.azure.account.key.casestudysk.dfs.core.windows.net", datalake_key)

In [0]:
df_swiggy = spark.read.format("delta").load('abfss://swiggy-silver@casestudysk.dfs.core.windows.net/swiggy-final/')

In [0]:
# df_swiggy.count()

In [0]:
# df_swiggy.orderBy("Restaurant_ID").display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

windowSpec = Window.partitionBy("Restaurant_ID").orderBy(col("update_timestamp").desc())

df_with_row_number = df_swiggy.withColumn("row_number", row_number().over(windowSpec))

df_swiggy = df_with_row_number.filter(col("row_number") == 1).drop("row_number")

df_swiggy.display()

In [0]:
spark.conf.set("fs.azure.account.key.casestudysk.dfs.core.windows.net", datalake_key)

delta_table_path_final = "abfss://swiggy-gold@casestudysk.dfs.core.windows.net/swiggy-gold-final/"

df_swiggy.write.format("delta").mode("overwrite").save(delta_table_path_final)